In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
import pycountry
from sklearn.preprocessing import LabelEncoder



import re

In [12]:
acquisition_df = pd.read_csv(r"E:\third-year sem2\machine- Copy\project\Acquisitions.csv")
acquired_df = pd.read_csv(r"E:\third-year sem2\machine- Copy\project\Acquired Tech Companies.csv")
acquiring_df = pd.read_csv(r"E:\third-year sem2\machine- Copy\project\Acquiring Tech Companies.csv")

Founders_df = pd.read_csv(r"Founders and Board Members.csv")


In [13]:
#acquired_df.head()

In [14]:
acquired_df.columns

Index(['Company', 'CrunchBase Profile', 'Image', 'Tagline', 'Year Founded',
       'Market Categories', 'Address (HQ)', 'City (HQ)', 'State / Region (HQ)',
       'Country (HQ)', 'Description', 'Homepage', 'Twitter', 'Acquired by',
       'Acquisitions ID', 'API'],
      dtype='object')

In [15]:
print(acquired_df.isnull().sum())


Company                  0
CrunchBase Profile       0
Image                   23
Tagline                  3
Year Founded            69
Market Categories       23
Address (HQ)            33
City (HQ)               35
State / Region (HQ)     37
Country (HQ)            34
Description             11
Homepage                37
Twitter                142
Acquired by              1
Acquisitions ID          0
API                      0
dtype: int64


In [16]:
print(acquired_df.duplicated().sum())

0


In [17]:
#print("status column unique values",acquisition_df["Status"].unique())
#print("Terms column unique values",acquisition_df["Terms"].unique())


In [18]:
acquired_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Company              310 non-null    object 
 1   CrunchBase Profile   310 non-null    object 
 2   Image                287 non-null    object 
 3   Tagline              307 non-null    object 
 4   Year Founded         241 non-null    float64
 5   Market Categories    287 non-null    object 
 6   Address (HQ)         277 non-null    object 
 7   City (HQ)            275 non-null    object 
 8   State / Region (HQ)  273 non-null    object 
 9   Country (HQ)         276 non-null    object 
 10  Description          299 non-null    object 
 11  Homepage             273 non-null    object 
 12  Twitter              168 non-null    object 
 13  Acquired by          309 non-null    object 
 14  Acquisitions ID      310 non-null    object 
 15  API                  310 non-null    obj

In [19]:
# task1 from the address get the country then remove the address




In [20]:

def extract_year(text):
    if not isinstance(text, str) or not text.strip():
        return None

    # Look for 'founded' followed by anything then a 4-digit year
    match = re.search(r'\bfounded\b.*?(\d{4})', text, re.IGNORECASE)
    if match:
        return int(match.group(1))
    
    return None


In [21]:
print(acquired_df['Year Founded'].isnull().sum())
# Apply the function to 'Tagline' and 'Description' where 'Year Founded' is null
acquired_df['Year Founded'] = acquired_df.apply(
    lambda row: extract_year(row['Tagline']) if pd.isnull(row['Year Founded']) else row['Year Founded'], axis=1
)

# If 'Year Founded' is still null, try using the Description column
acquired_df['Year Founded'] = acquired_df.apply(
    lambda row: extract_year(row['Description']) if pd.isnull(row['Year Founded']) else row['Year Founded'], axis=1
)
print(acquired_df['Year Founded'].isnull().sum())

69
57


In [22]:

'''acquired_df['Year Founded'].fillna(acquired_df['Year Founded'].median(), inplace=True)
print(acquired_df['Year Founded'].isnull().sum())'''



"acquired_df['Year Founded'].fillna(acquired_df['Year Founded'].median(), inplace=True)\nprint(acquired_df['Year Founded'].isnull().sum())"

In [23]:
# --- Step 4.5: Fill missing 'Market Categories' from 'Tagline' ---

# Fill NaN values with 'Unknown'
acquired_df['Market Categories'] = acquired_df['Market Categories'].fillna('Unknown')

# Define keyword-based categories
category_keywords = {
    "Artificial Intelligence": ["ai", "machine learning", "deep learning", "neural network"],
    "Mobile": ["mobile", "android", "ios", "app store", "smartphone"],
    "E-Commerce": ["ecommerce", "e-commerce", "shopping", "online store"],
    "FinTech": ["finance", "banking", "payments", "fintech", "crypto", "blockchain"],
    "Healthcare": ["health", "medical", "hospital", "doctor", "pharma"],
    "Social Media": ["social network", "community", "messaging", "chat"],
    "Gaming": ["game", "gaming", "video game", "esports"],
    "Cloud": ["cloud", "saas", "paas", "infrastructure"],
    "EdTech": ["education", "learning", "students", "teaching", "school"],
    "Data Analytics": ["analytics", "data science", "big data", "insights"]
}

# Optional: preview unique existing categories
split_categories = acquired_df['Market Categories'].apply(lambda x: [i.strip() for i in x.split(',') if i.strip()])
all_categories = sorted(set(cat for sublist in split_categories for cat in sublist))
print("Sample existing categories:", all_categories[:20])
print("Total unique categories:", len(all_categories))

# Function to guess category from tagline

'''def guess_category_from_tagline(tagline):
    tagline = str(tagline).lower()
    matched = [cat for cat, keywords in category_keywords.items()
               if any(keyword in tagline for keyword in keywords)]
    return ', '.join(matched) if matched else "Unknown"'''


def guess_category_from_tagline(tagline):
    tagline = str(tagline).lower()
    matched = [cat for cat, keywords in category_keywords.items()
               if any(keyword in tagline for keyword in keywords)]
    
    # Ensure at least 2 values are returned
    if len(matched) == 0:
        matched = ["Software", "Advertising"]
    elif len(matched) == 1:
        matched.append("Software")
    
    return ', '.join(matched)
# Fill only where category is unknown
acquired_df['Market Categories'] = acquired_df.apply(
    lambda row: guess_category_from_tagline(row['Tagline']) if row['Market Categories'] in ["Unknown", "nan", "", None] else row['Market Categories'],
    axis=1
)


Sample existing categories: ['3D', 'Advertising', 'Aerospace', 'Analytics', 'Android', 'Apps', 'Auctions', 'B2B', 'Banking', 'Big Data', 'Browser Extensions', 'CRM', 'Cable', 'Career Management', 'Chat', 'Clean Technology', 'Clinical Trials', 'Cloud Computing', 'Cloud Data Services', 'Cloud Security']
Total unique categories: 153


In [24]:
# Define the mapping of specific categories to generalized categories
category_mapping = {
    'Software': 'Technology & Software',
    'Advertising': 'Advertising & Marketing',
    'E-Commerce': 'E-Commerce & Online Services',
    'Mobile': 'Mobile & Consumer Electronics',
    'Games': 'Games & Entertainment',
    'Social Media': 'Social Networking & Communication',
    'Cloud': 'Technology & Software',
    'Finance': 'Finance & Payments',
    'Healthcare': 'Healthcare & Wellness',
    'Semiconductors': 'Technology Hardware',
    'Data Analytics': 'Analytics & Data Science',
    'Search': 'Advertising & Marketing',
    'Video': 'Games & Entertainment',
    'Networking': 'Telecom & Networks',
    'Messaging': 'Social Networking & Communication',
    'Education': 'Education & Learning',
    'News': 'Media & News',
    'Photo Sharing': 'Digital Media & Content',
    'Mobile Payments': 'Finance & Payments',
    'Robotics': 'Games & Entertainment',
    'Music': 'Games & Entertainment',
    'Photo Editing': 'Digital Media & Content',
    'Online Rental': 'E-Commerce & Online Services',
    'Location Based Services': 'Telecom & Networks',
    'Enterprise Software': 'Technology & Software',
    'Video Streaming': 'Games & Entertainment',
    'PaaS': 'Technology & Software',
    'SaaS': 'Technology & Software',
    'Health and Wellness': 'Healthcare & Wellness',
    'Web Hosting': 'Technology & Software',
    'Internet of Things': 'IoT (Internet of Things)',
    'Cloud Security': 'Technology & Software',
    'Virtual Currency': 'Finance & Payments',
    'Search Marketing': 'Advertising & Marketing',
    'Mobile Social': 'Social Networking & Communication',
    'Retail': 'Retail & Fashion',
    'Consulting': 'Others & Miscellaneous',
    'Aerospace': 'Others & Miscellaneous',
    'Food Delivery': 'Consumer Goods & Services',
    'Fashion': 'Retail & Fashion',
    'Wine And Spirits': 'Consumer Goods & Services',
    'Streaming': 'Games & Entertainment',
    'Task Management': 'Others & Miscellaneous',
    'Video Chat': 'Social Networking & Communication',
    'Personalization': 'Advertising & Marketing',
    'Shopping': 'E-Commerce & Online Services',
    'Local': 'E-Commerce & Online Services',
    'News': 'Media & News',
    'Fraud Detection': 'Advertising & Marketing',
    'Image Recognition': 'Technology Hardware',
    'Virtualization': 'Games & Entertainment',
    'Analytics': 'Analytics & Data Science',
    'Video on Demand': 'Games & Entertainment',
    'Mobile Payments': 'Finance & Payments',
    'Marketing Automation': 'Advertising & Marketing',
    'Consumer Electronics': 'Mobile & Consumer Electronics',
    'Video Games': 'Games & Entertainment',
    'Public Relations': 'Advertising & Marketing'
}

# Add any other specific categories to the mapping above


In [25]:
# Function to map specific categories to the generalized ones
def map_to_generalized_category(row):
    # Split the current market categories and map each category
    categories = row.split(',')  # Split by comma
    generalized_categories = []

    for category in categories:
        category = category.strip()  # Remove any extra spaces
        if category in category_mapping:
            generalized_categories.append(category_mapping[category])
        else:
            generalized_categories.append('Others & Miscellaneous')  # In case of unknown categories

    return ', '.join(set(generalized_categories))  # Return unique generalized categories as a string

# Apply the mapping function to the 'Market Categories' column
acquired_df['Generalized Market Categories'] = acquired_df['Market Categories'].apply(map_to_generalized_category)

# Print the full output without truncation
print(acquired_df[['Market Categories', 'Generalized Market Categories']].to_string(index=False))


                                                                                                            Market Categories                                                                                                         Generalized Market Categories
                                                                                                                     Software                                                                                                                 Technology & Software
                                                                                                                  Advertising                                                                                                               Advertising & Marketing
                                                                                                                     Software                                                                                               

In [26]:
acquired_df.loc[acquired_df['Company']=="Emagic"]

,Company,CrunchBase Profile,Image,Tagline,Year Founded,Market Categories,Address (HQ),City (HQ),State / Region (HQ),Country (HQ),Description,Homepage,Twitter,Acquired by,Acquisitions ID,API,Generalized Market Categories
30,Emagic,http://www.crunchbase.com/organization/emagic,NaN,Emagic was a germany-based a provider of music...,NaN,"Software, Advertising","250 E. Kilbourn Avenue, Milwaukee, Wisconsin, ...",Milwaukee,Wisconsin,United States,Emagic was a germany-based a provider of music...,NaN,NaN,Apple,Apple acquired Emagic in 2002,http://api.crunchbase.com/v/2/organization/ema...,"Technology & Software, Advertising & Marketing"


In [27]:
acquired_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Company                        310 non-null    object 
 1   CrunchBase Profile             310 non-null    object 
 2   Image                          287 non-null    object 
 3   Tagline                        307 non-null    object 
 4   Year Founded                   253 non-null    float64
 5   Market Categories              310 non-null    object 
 6   Address (HQ)                   277 non-null    object 
 7   City (HQ)                      275 non-null    object 
 8   State / Region (HQ)            273 non-null    object 
 9   Country (HQ)                   276 non-null    object 
 10  Description                    299 non-null    object 
 11  Homepage                       273 non-null    object 
 12  Twitter                        168 non-null    obj

In [28]:
#moh
def extract_city_region_clean(row):
    if all(pd.isnull(row[col]) for col in ["Address (HQ)", "City (HQ)", "State / Region (HQ)", "Country (HQ)"]):
        desc = row.get("Description", "")
        if isinstance(desc, str):
            match = re.search(r'\b(founded in|based in|headquarters in)\s+([^,]+),\s*([^\s,]+)', desc, re.IGNORECASE)
            if match:
                city = match.group(2).strip()
                region_raw = match.group(3).strip()
                # Remove all non-letter characters from region
                region = re.sub(r'[^A-Za-z]', '', region_raw)
                row["City (HQ)"] = city
                row["State / Region (HQ)"] = region
    return row


acquired_df = acquired_df.apply(extract_city_region_clean, axis=1)
acquired_df.isnull().sum()


Company                            0
CrunchBase Profile                 0
Image                             23
Tagline                            3
Year Founded                      57
Market Categories                  0
Address (HQ)                      33
City (HQ)                         28
State / Region (HQ)               30
Country (HQ)                      34
Description                       11
Homepage                          37
Twitter                          142
Acquired by                        1
Acquisitions ID                    0
API                                0
Generalized Market Categories      0
dtype: int64

In [29]:
# Create a set of country names for fast lookup
all_countries = {country.name.lower(): country.name for country in pycountry.countries}


def extract_country_without_regex(row):
    if pd.isna(row["Country (HQ)"]):
        description = str(row.get("Description", ""))  # Ensure we have a string
        
        # Common country name variations to check
        country_phrases = [
            " in ", " based in ", " headquartered in ", 
            " located in ", " from ", " of "
        ]
        
        # Check each phrase pattern
        for phrase in country_phrases:
            if phrase in description.lower():
                # Get the text after the phrase
                after_phrase = description.lower().split(phrase)[-1]
                
                # Try to find a country name in the next few words
                for word in after_phrase.split()[:4]:  # Check next 4 words max
                    word = word.strip(".,;")  # Clean punctuation
                    if word in all_countries:
                        row["Country (HQ)"] = all_countries[word]
                        return row  # Return after first valid match
        
        # Check for standalone country mentions
        for word in description.split():
            word = word.strip(".,;").lower()
            if word in all_countries:
                row["Country (HQ)"] = all_countries[word]
                break
    
    return row

# Example usage with a DataFrame:
# df = df.apply(extract_country_without_regex, axis=1)


In [30]:
acquired_df = acquired_df.apply(extract_country_without_regex, axis=1)
acquired_df.isnull().sum()

Company                            0
CrunchBase Profile                 0
Image                             23
Tagline                            3
Year Founded                      57
Market Categories                  0
Address (HQ)                      33
City (HQ)                         28
State / Region (HQ)               30
Country (HQ)                      31
Description                       11
Homepage                          37
Twitter                          142
Acquired by                        1
Acquisitions ID                    0
API                                0
Generalized Market Categories      0
dtype: int64

In [31]:
def create_country_region_dict_preserve_nulls(df):
    """
    Creates a dictionary mapping countries to their regions exactly as they appear in the data.
    Preserves null values and does no cleaning or transformation, except for fixing "United Stats of America".
    Also adds common regions for major countries if they're not already in the data.
    Returns: {country: [list of regions]} (including nulls if present)
    """
    country_region_dict = {}
    
    # Common regions for major countries (to add if not present)
    common_regions = {
        'United States': ['CA','California', 'New York', 'Texas', 'Florida', 'Illinois', 'Pennsylvania', 'Ohio', 'Georgia', 'North Carolina', 'Michigan'],
        'Canada': ['Ontario', 'Quebec', 'British Columbia', 'Alberta', 'Manitoba'],
        'United Kingdom': ['England', 'Scotland', 'Wales', 'Northern Ireland'],
        'Germany': ['Bavaria', 'Berlin', 'Hamburg', 'North Rhine-Westphalia', 'Baden-Württemberg'],
        'France': ['Île-de-France', 'Auvergne-Rhône-Alpes', 'Nouvelle-Aquitaine', 'Occitanie', 'Hauts-de-France'],
        'China': ['Beijing', 'Shanghai', 'Guangdong', 'Jiangsu', 'Zhejiang'],
        'Japan': ['Tokyo', 'Osaka', 'Kanagawa', 'Aichi', 'Hokkaido'],
        'Australia': ['New South Wales', 'Victoria', 'Queensland', 'Western Australia', 'South Australia'],
        'India': ['Maharashtra', 'Delhi', 'Karnataka', 'Tamil Nadu', 'Uttar Pradesh'],
        'Brazil': ['São Paulo', 'Rio de Janeiro', 'Minas Gerais', 'Rio Grande do Sul', 'Bahia']
    }
    
    for country, region in zip(df['Country (HQ)'], df['State / Region (HQ)']):
        # Skip if either value is null
        if pd.isna(country) or pd.isna(region):
            continue
            
        # Fix common typo
        normalized_country = "United States" if country.lower() in ["united stats of america", "united stats"] else country
            
        # Initialize country entry if not exists
        if normalized_country not in country_region_dict:
            country_region_dict[normalized_country] = []
            
        # Add region if not already present
        if region not in country_region_dict[normalized_country]:
            country_region_dict[normalized_country].append(region)
    
    # Add common regions for countries that are in our data but might be missing regions
    for country, regions in common_regions.items():
        if country in country_region_dict:
            # Add regions that aren't already present
            existing_regions = set(country_region_dict[country])
            new_regions = [r for r in regions if r not in existing_regions]
            country_region_dict[country].extend(new_regions)
        elif country in df['Country (HQ)'].values:
            # Country exists in data but not in our dict (probably all null regions)
            country_region_dict[country] = regions
    
    return country_region_dict

In [32]:
country_region_mapping = create_country_region_dict_preserve_nulls(acquired_df)

print(country_region_mapping)

{'Switzerland': ['Basel-Stadt', 'Geneve', 'Thurgau'], 'United States': ['California', 'Utah', 'New Jersey', 'Massachusetts', 'Texas', 'New York', 'Maryland', 'Pennsylvania', 'Illinois', 'Wisconsin', 'Indiana', 'District of Columbia', 'Vermont', 'Washington', 'Virginia', 'Minnesota', 'Florida', 'Colorado', 'Georgia', 'Arkansas', 'Tennessee', 'CA', 'Ohio', 'North Carolina', 'Michigan'], 'France': ['Ile-de-France', 'Île-de-France', 'Auvergne-Rhône-Alpes', 'Nouvelle-Aquitaine', 'Occitanie', 'Hauts-de-France'], 'Great Britain': ['United Kingdom', 'Belfast', 'Reading', 'Gloucestershire'], 'Sweden': ['Skane Lan', 'Stockholms Lan'], 'Israel': ['Tel Aviv', 'HaMerkaz', 'Yerushalayim', 'Hefa'], 'Canada': ['Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta', 'Manitoba'], 'Germany': ['Bayern', 'Sachsen', 'Bavaria', 'Berlin', 'Hamburg', 'North Rhine-Westphalia', 'Baden-Württemberg'], 'Japan': ['Gumma', 'Kyushu', 'Tokyo', 'Osaka', 'Kanagawa', 'Aichi', 'Hokkaido'], 'Denmark': ['Syddanmar

In [33]:
def fill_country_from_region(df, country_region_dict):
    """
    Fills null Country (HQ) values based on State/Region (HQ) using a predefined mapping.
    Only fills if:
    1. Country is null
    2. Region exists and is not null
    3. Region maps to exactly one country in the dictionary
    """
    # Create reverse mapping (region -> country)
    region_country_map = {}
    for country, regions in country_region_dict.items():
        for region in regions:
            # Handle case where region maps to multiple countries
            if region in region_country_map:
                region_country_map[region] = None  # Mark ambiguous regions
            else:
                region_country_map[region] = country
    
    # Fill null countries where we have unambiguous region mapping
    for idx, row in df.iterrows():
        if pd.isna(row['Country (HQ)']) and not pd.isna(row['State / Region (HQ)']):
            region = row['State / Region (HQ)']
            if region in region_country_map and region_country_map[region] is not None:
                df.at[idx, 'Country (HQ)'] = region_country_map[region]
    
    return df

In [34]:
# Then use it to fill countries
acquired_df = fill_country_from_region(acquired_df, country_region_mapping)
acquired_df.isnull().sum()

Company                            0
CrunchBase Profile                 0
Image                             23
Tagline                            3
Year Founded                      57
Market Categories                  0
Address (HQ)                      33
City (HQ)                         28
State / Region (HQ)               30
Country (HQ)                      25
Description                       11
Homepage                          37
Twitter                          142
Acquired by                        1
Acquisitions ID                    0
API                                0
Generalized Market Categories      0
dtype: int64

In [35]:
columns_to_drop = ['Image', 'CrunchBase Profile', 'Homepage', 'Twitter','Address (HQ)','API','Description','Tagline','Market Categories']
acquired_df.drop(columns=columns_to_drop, inplace=True)
print(acquired_df.columns)



Index(['Company', 'Year Founded', 'City (HQ)', 'State / Region (HQ)',
       'Country (HQ)', 'Acquired by', 'Acquisitions ID',
       'Generalized Market Categories'],
      dtype='object')


In [36]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Fill nulls in categorical columns using mode
for col in ['City (HQ)', 'State / Region (HQ)', 'Country (HQ)']:
    mode_value = acquired_df[col].mode()[0]
    acquired_df[col].fillna(mode_value, inplace=True)

# Label Encoding
label_encoders = {}
for col in ['City (HQ)', 'State / Region (HQ)', 'Country (HQ)']:
    le = LabelEncoder()
    acquired_df[col + '_LabelEncoded'] = le.fit_transform(acquired_df[col].astype(str))
    label_encoders[col] = le

# One-hot encoding
df_one_hot = pd.get_dummies(acquired_df[['City (HQ)', 'State / Region (HQ)', 'Country (HQ)']],
                            prefix=['City', 'State', 'Country'])

# Drop the original columns
acquired_df.drop(columns=['City (HQ)', 'State / Region (HQ)', 'Country (HQ)'], inplace=True)

# Combine everything
df_final = pd.concat([acquired_df, df_one_hot], axis=1)

# View the result
print(df_final.head())

              Company  Year Founded Acquired by  \
0        Day Software        1993.0       Adobe   
1  Efficient Frontier        2002.0       Adobe   
2          Macromedia        1992.0       Adobe   
3             Neolane        2001.0       Adobe   
4            Omniture        1996.0       Adobe   

                                     Acquisitions ID  \
0        Adobe Systems acquired Day Software in 2010   
1  Adobe Systems acquired Efficient Frontier in 2011   
2          Adobe Systems acquired Macromedia in 2005   
3             Adobe Systems acquired Neolane in 2013   
4            Adobe Systems acquired Omniture in 2009   

                    Generalized Market Categories  City (HQ)_LabelEncoded  \
0                           Technology & Software                      10   
1                         Advertising & Marketing                     105   
2                           Technology & Software                      94   
3                           Technology & Softwar

C:\Users\www\AppData\Local\Temp\ipykernel_576\266746161.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  acquired_df[col].fillna(mode_value, inplace=True)


In [37]:
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

# Step 1: Convert the 'Generalized Market Categories' column to a list of categories
acquired_df['MultiCategoryList'] = acquired_df['Generalized Market Categories'].apply(
    lambda x: [cat.strip() for cat in x.split(',')] if pd.notnull(x) else []
)

# Step 2: Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Step 3: Transform the list column to one-hot encoded format
multi_hot_encoded = mlb.fit_transform(acquired_df['MultiCategoryList'])

# Step 4: Convert to DataFrame with proper column names
df_multi_hot = pd.DataFrame(multi_hot_encoded, columns=mlb.classes_, index=acquired_df.index)

# Step 5: Drop the helper column
acquired_df.drop(columns=['MultiCategoryList'], inplace=True)

# Step 6: Merge back the encoded columns
acquired_df = pd.concat([acquired_df, df_multi_hot], axis=1)

# Final result
print(acquired_df.head())


              Company  Year Founded Acquired by  \
0        Day Software        1993.0       Adobe   
1  Efficient Frontier        2002.0       Adobe   
2          Macromedia        1992.0       Adobe   
3             Neolane        2001.0       Adobe   
4            Omniture        1996.0       Adobe   

                                     Acquisitions ID  \
0        Adobe Systems acquired Day Software in 2010   
1  Adobe Systems acquired Efficient Frontier in 2011   
2          Adobe Systems acquired Macromedia in 2005   
3             Adobe Systems acquired Neolane in 2013   
4            Adobe Systems acquired Omniture in 2009   

                    Generalized Market Categories  City (HQ)_LabelEncoded  \
0                           Technology & Software                      10   
1                         Advertising & Marketing                     105   
2                           Technology & Software                      94   
3                           Technology & Softwar

In [39]:
acquired_df.isnull().sum()

Company                               0
Year Founded                         57
Acquired by                           1
Acquisitions ID                       0
Generalized Market Categories         0
City (HQ)_LabelEncoded                0
State / Region (HQ)_LabelEncoded      0
Country (HQ)_LabelEncoded             0
Advertising & Marketing               0
Analytics & Data Science              0
Consumer Goods & Services             0
Digital Media & Content               0
E-Commerce & Online Services          0
Education & Learning                  0
Finance & Payments                    0
Games & Entertainment                 0
Healthcare & Wellness                 0
IoT (Internet of Things)              0
Media & News                          0
Mobile & Consumer Electronics         0
Others & Miscellaneous                0
Retail & Fashion                      0
Social Networking & Communication     0
Technology & Software                 0
Technology Hardware                   0


In [38]:

acquired_df.columns


Index(['Company', 'Year Founded', 'Acquired by', 'Acquisitions ID',
       'Generalized Market Categories', 'City (HQ)_LabelEncoded',
       'State / Region (HQ)_LabelEncoded', 'Country (HQ)_LabelEncoded',
       'Advertising & Marketing', 'Analytics & Data Science',
       'Consumer Goods & Services', 'Digital Media & Content',
       'E-Commerce & Online Services', 'Education & Learning',
       'Finance & Payments', 'Games & Entertainment', 'Healthcare & Wellness',
       'IoT (Internet of Things)', 'Media & News',
       'Mobile & Consumer Electronics', 'Others & Miscellaneous',
       'Retail & Fashion', 'Social Networking & Communication',
       'Technology & Software', 'Technology Hardware', 'Telecom & Networks'],
      dtype='object')